In [ ]:
!pip install -U skorch

In [ ]:
import numpy as np
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
from skorch import NeuralNetClassifier

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from cuml.preprocessing.TargetEncoder import TargetEncoder
from tqdm.notebook import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-mar-2021/train.csv')
test  = pd.read_csv('../input/tabular-playground-series-mar-2021/test.csv')
sub = pd.read_csv('../input/tabular-playground-series-mar-2021/sample_submission.csv')

In [ ]:
train.columns

In [ ]:
target = train['target'].values
columns = test.columns[1:]
cat_features = columns[:19]
cat_features

In [ ]:
class ClassificationModule(nn.Module):
    def __init__(self,num_units=100, nonlin=F.relu):
        
        super(ClassificationModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin

        self.dense0 = nn.Linear(len(columns), num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(0.2)
        self.dense1 = nn.Linear(num_units, 20)
        self.output = nn.Linear(20, 2)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = F.relu(self.dense1(X))
        X = F.softmax(self.output(X))
        return X

In [ ]:
%%time

train_oof = np.zeros((300000,))
test_preds = 0
train_oof.shape

NUM_FOLDS = 10
kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=137)
NUM_EPOCHS = 20

for f, (train_ind, val_ind) in tqdm(enumerate(kf.split(train, target))):
        #print(f'Fold {f}')
        train_df, val_df = train.iloc[train_ind][columns], train.iloc[val_ind][columns]
        train_target, val_target = target[train_ind], target[val_ind]
        test_df = test.copy()
        
        for cat_col in cat_features:
            te = TargetEncoder()
            train_df[cat_col] = te.fit_transform(train_df[cat_col], train_target)
    
            val_df[cat_col] = te.transform(val_df[cat_col])
            test_df[cat_col] = te.transform(test_df[cat_col])
            
            
        model =  NeuralNetClassifier(ClassificationModule, lr=0.01, max_epochs=NUM_EPOCHS, optimizer=optim.Adam, device='cuda')
        X = train_df.values.astype(np.float32)
        y = train_target.astype(np.int64)
        model.fit(X, y)
        temp_oof = model.predict_proba(val_df.values.astype(np.float32))[:,1].flatten()
        temp_test = model.predict_proba(test_df[columns].values.astype(np.float32))[:,1].flatten()

        train_oof[val_ind] = temp_oof
        test_preds += temp_test/NUM_FOLDS
        
        print(roc_auc_score(val_target, temp_oof))
            
    

In [ ]:
print('\nOverall score:', roc_auc_score(target, train_oof))

In [ ]:
sub['target'] = test_preds
sub.to_csv('submission_nn.csv', index=False)
sub.head()